
# How to generate text: using different decoding methods for language generation with Transformers

### **Llama 3.2 3B (2024년 9월 발표)**

#### ***Language Modeling***
- OpenAI사의 GPT 모델과 마찬가지로 transformer decoder 블럭으로 구성되어 있고, auto-regressive language modeling으로 학습 됨
- 기존 LLaMA 모델들의 아키텍처를 기반으로 개선된 attention 메커니즘 도입
- 더 효율적인 학습 및 추론을 위해 적응형 최적화 기술과 함께 큰 context window 지원

#### ***Various Training Dataset***
- Open-source 데이터뿐만 아니라 학문적 논문, 코드 스니펫, 웹 텍스트 등 다양한 도메인에서 추출된 고품질 데이터셋으로 학습
- 데이터 클리닝과 필터링 기술을 통해 노이즈 감소, 더욱 신뢰도 높은 언어 모델링 구현

#### ***Tokenizer***
- LLaMA 3.2는 개선된 Byte Pair Encoding 기반의 [토크나이저](https://github.com/openai/tiktoken)를 사용하여 더 많은 언어와 문법 구조 지원
- 어휘 크기를 32,000개에서 128,000개로 확장해 토큰 효율성이 최대 15% 향상되어, 동일한 텍스트를 표현하는 데 필요한 토큰 수가 줄어들어 추론 속도와 비용 효율성 개선.

#### ***Zero-shot 및 Few-shot Learning***
- Fine-tuning 없이도 다양한 작업에 뛰어난 Zero-shot 성능 발휘
- 특히 3B 모델은 가벼운 크기로도 Few-shot 학습에서 효율성과 성능 모두 보장

### **Introduction**

최근, 대량의 weppage 데이터로 학습 된 transformer-based language models의 등장으로 open-ended language generation에 대한 관심이 높아졌습니다.

향상된 tranformer architecure와 대량의 unsupervised trainig data외에도 더 나은 decoding methods 또한 중요한 역할을 했습니다.

본 자료에서는 다양한 decoding strategies에 대한 간략한 개요와  `transformers` library를 사용해 손쉽게 이러한 decoding strategies를 구현하는 방법을 다룹니다.

모든 방법들은 **auto-regressive** language generation ([참고 자료](http://jalammar.github.io/illustrated-gpt2/)) 를 사용합니다.

![GPT-2 Auto regressive](http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif)

(그림은 GPT-2라고 적혀있지만, 다른 대부분의 언어 모델도 마찬가지입니다.)

*Auto-regressive* language generation는 word sequence의 확률은 next word의 conditional distribution으로 decompose할 수 있다는 것이 기본 가정입니다.
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$

$W_0$는 initial *context* word sequence를 의미합니다. Word sequence의 길이인 $T$는 $P(w_{t} | w_{1: t-1}, W_{0})$으로 부터 EOS token이 나온 timestep $t=T$를 의미합니다.

이번 실습 시간에는 현재 유명한 decoding 방법들인 *Greedy search*, *Beam search*, *Top-K sampling*, *Top-p sampling* 를 다룹니다.

Transformer를 설치하고 Model을 load 해봅시다.

이번 실습 시간에서는 Bllossom에서 공개한, 한국어로 학습된 `llama-3.2-Korean-Bllossom-3B` 모델을 사용합니다.


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
set_seed(42)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### **Greedy Search**

Greedy search는 다음 단어를 선택할 때, 가장 높은 확률을 가진 단어를 선택하는 단순한 방법입니다.

$w_t = argmax_{w}P(w | w_{1:t-1})$ at each timestep $t$.

다음 그림은 greedy search를 나타낸 것 입니다.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)


알고리즘은 $\text{"The"}$에서 시작하여 가장 높은 확률을 가진 $\text{"nice"}$ 등을 선택하는 탐욕스러운(Greedy) 방법입니다. 따라서 최종적으로 생성 된 word sequence는 $\text{"The", "nice", "woman"}$이고 전체 확률은 $0.5 \times 0.4 = 0.2$ 입니다.

`Transformers` greedy search를 사용해봅시다.

In [9]:
# encode context the generation is conditioned on
def tokenizing(text):
    messages = [
        {"role": "user", "content": f"{text}"}
        ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    return input_ids

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizing("오늘 점심엔 뭘 먹을까요")

# generate text until the output length (which includes the context length) reaches 100
# 생성 모델은 generate 함수를 통해 다음 token을 생성해낼 수 있습니다.
# 그냥 넣어주면 자동으로 greedy search를 시작.
print(input_ids)
greedy_output = model.generate(input_ids,
                               max_new_tokens=512,
                               eos_token_id=terminators,
                               do_sample=False,
                               num_beams=1)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0][input_ids.shape[-1]:], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1544,   4723,    220,   2366,     19,    271,   2675,    527,
            264,  11190,  15592,  18328,     13,   5321,   4320,    279,   1217,
            596,   4860,  47626,     13, 113783,  34804, 101003,  67119,  24486,
          15592, 101139,  30426,  25941,  95252,  29726, 119519,     13,  41820,
         110257, 109760,  19954, 112107, 108280, 104834, 102893, 111964,  34983,
          92769,     13, 128009, 128006,    882, 128007,    271,  58368, 105622,
         106313, 102612, 108733,   5251,    255,    246, 108715,  18359, 117677,
         128009, 128006,  78191, 128007,    271]], device='cuda:0')
Output:
----------------------------------------------------------------------------------------------------
오늘 점심을 선택할 때는 여러 가지 요소를 고려해볼 수 있습니다. 예를 들어, 기분, 재료, 시간, 예산 등이 중요할 수 있습니다. 다음은 몇 가지 아이디어입니다:

1

Llama 3.2 3B를 사용해 짧은 텍스트를 생성했습니다.

Greedy Search의 주요 단점은 그림에서 볼 수 있듯이 낮은 확률 단어 이후에 나올 수 있는 더 높은 확률의 단어를 놓친다는 점입니다.

예를 들어 단어 $\text{has}$는 0.9로 높은 확률을 갖지만 첫번째 단어 후보 중 두번째로 높은 conditional probability를 갖는 $\text{dog}$ 이후에 숨어 있는 형태입니다. 따라서 Greedy Search는 $\text{"The"}, \text{"dog"}, \text{"has"}$ 라는 word sequence를 놓치게 됩니다.

### **Beam search**

Beam search는 각 time step에서 가장 확률이 높은 hypothesis의 `num_beams`를 유지하고 전체 확률이 가장 높은 hypothesis를 선택하는 방법입니다. 즉, 해당 시점에서 유망한 빔의 개수만큼(num_beams) 골라서 진행하는 방식으로 높은 확률을 가지고 있지만 숨어있는 word sequence를 놓칠 위험을 줄입니다.다음 그림은 `num_beams=2`로 설정한 Beam search의 예시입니다:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)


Time step $1$ : 가장 가능성이 높은 hypothesis인 $\text{"The", "nice"}$ 외에도 beam search는 두번째로 가능성이 높은 $\text{"The", "dog"}$를 추적합니다.

Time step $2$ : beam search는 $0.2$의 가능성을 가진  $\text{"The", "nice", "woman"}$보다 $0.36$의 가능성을 가진 $\text{"The", "dog", "has"}$가 확률이 더 높다는 것을 찾습니다.

이 방법은 우리의 toy example에서 가장 가능성이 높은 word sequence를 찾아냈습니다.

Beam search는 항상 Greedy search보다 높은 확률의 결과 sequence를 찾는 것이 가능하지만, 이게 가장 가능성이 높은 결과라고는 보장할 수 없습니다.

`transformers`에서 beam search를 사용하는 방법을 살펴 봅시다. 우리는 `num_beams > 1`, `early_stopping=True` 으로 설정하여 모든 beam hypothesis가 eos토큰에 닿으면 생성을 마치도록 합니다.

In [10]:
# activate beam search and early_stopping
beam_output = model.generate(input_ids,
                             max_new_tokens=512,
                             eos_token_id=terminators,
                             do_sample=False,
                             num_beams=3,
                             early_stopping=True)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
오늘 점심을 선택할 때는 여러 가지 요소를 고려해볼 수 있습니다. 예를 들어, 기호, 재료, 예산, 건강성 등이 중요한 요소가 될 수 있습니다. 몇 가지 제안을 드릴게요:

1. **비빔밥**: 비빔밥은 다양한 재료를 넣어 먹을 수 있어 기호에 맞게 조절할 수 있습니다. 예를 들어, 고기, 채소, 과일 등을 넣어 먹을 수 있습니다.

2. **비빔밥과 함께 먹는 요리**: 비빔밥과 함께 먹을 수 있는 요리도 좋은 선택이 될 수 있습니다. 예를 들어, 비빔밥과 함께 먹을 수 있는 고기 요리나 채소 요리 등이 있습니다.

3. **비빔밥과 함께 먹을 수 있는 간식**: 비빔밥과 함께 먹을 수 있는 간식도 좋은 선택이 될 수 있습니다. 예를 들어, 과일, 쿠키 등이 있습니다.

4. **비빔밥과 함께 먹을 수 있는 음료**: 비빔밥과 함께 먹을 수 있는 음료도 좋은 선택이 될 수 있습니다. 예를 들어, 차, 커피 등이 있습니다.

5. **비빔밥과 함께 먹을 수 있는 과일**: 비빔밥과 함께 먹을 수 있는 과일도 좋은 선택이 될 수 있습니다. 예를 들어, 바나나, 사과 등이 있습니다.

이러한 제안을 바탕으로 오늘 점심을 선택해 보세요!


Beam search를 적용하면 조금 더 유창해 지는 것을 보실 수 있습니다.

Beam search의 또 다른 중요한 특징은 생성된 Top beam을 비교하여 목적에 가장 적합한 Beam을 선택할 수 있다는 것입니다.

`Transformers`에서 우리는 `num_return_sequences`를 top-n개의 높은 scoring을 가진 beam을 return할 것인지 설정 할 수 있습니다.

단, num_return_sequences는 num_beams보다 같거나 작아야합니다.

In [16]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=False,
    num_beams=3,
    early_stopping=True,
    num_return_sequences=3,
)

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
  print("<Output {}>\n{}\n".format(i, tokenizer.decode(beam_output[input_ids.shape[-1]:], skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<Output 0>
오늘 점심을 선택할 때는 여러 가지 요소를 고려해볼 수 있습니다. 예를 들어, 기호, 재료, 예산, 건강성 등이 중요한 요소가 될 수 있습니다. 몇 가지 제안을 드릴게요:

1. **비빔밥**: 비빔밥은 다양한 재료를 넣어 먹을 수 있어 기호에 맞게 조절할 수 있습니다. 예를 들어, 고기, 채소, 과일 등을 넣어 먹을 수 있습니다.

2. **비빔밥과 함께 먹는 요리**: 비빔밥과 함께 먹을 수 있는 요리도 좋은 선택이 될 수 있습니다. 예를 들어, 비빔밥과 함께 먹을 수 있는 고기 요리나 채소 요리 등이 있습니다.

3. **비빔밥과 함께 먹을 수 있는 간식**: 비빔밥과 함께 먹을 수 있는 간식도 좋은 선택이 될 수 있습니다. 예를 들어, 과일, 쿠키 등이 있습니다.

4. **비빔밥과 함께 먹을 수 있는 음료**: 비빔밥과 함께 먹을 수 있는 음료도 좋은 선택이 될 수 있습니다. 예를 들어, 차, 커피 등이 있습니다.

5. **비빔밥과 함께 먹을 수 있는 과일**: 비빔밥과 함께 먹을 수 있는 과일도 좋은 선택이 될 수 있습니다. 예를 들어, 바나나, 사과 등이 있습니다.

이러한 제안을 바탕으로 오늘 점심을 선택해 보세요!

<Output 1>
오늘 점심을 선택할 때는 여러 가지 요소를 고려해볼 수 있습니다. 예를 들어, 기호, 재료, 예산, 건강성 등이 중요한 요소가 될 수 있습니다. 몇 가지 제안을 드릴게요:

1. **비빔밥**: 비빔밥은 다양한 재료를 넣어 먹을 수 있어 기호에 맞게 조절할 수 있습니다. 예를 들어, 고기, 채소, 과일 등을 넣어 먹을 수 있습니다.

2. **비빔밥과 함께 먹는 요리**: 비빔밥과 함께 먹을 수 있는 요리도 좋은 선택이 될 수 있습니다. 예를 들어, 비빔밥과 함께 먹을 수 있는 고기 요리나 채소 요리 등이 있습니다.

3. **비빔밥과 함께 먹을 수 있는 간식**: 비빔밥과 함께 먹을 수 있는 간식도 좋은 선택이 될 수 있습니다. 예를 들어, 과일, 쿠키 등이 있습니다.

4. **비빔밥과 

위에서 볼 수 있듯이 Top 3의 Beam hypothesis는 서로 약간만 다를 뿐이며 3개만 사용했을 경우 별로 놀랄만한 결과는 아닙니다.

Open-ended generation에서, beam search가 최선의 선택이 아닐 수 있는 몇가지 이유가 제시 되었습니다.

- Beam search는 Machine translation 또는 Text summarization처럼 원하는 문장 생성 길이가 예측 가능한 task에서는 잘 작동 될 수 있습니다. 하지만 Dialog 또는 Story generation task처럼 출력 길이가 크게 달라질 수 있는 open-ended generation에서는 원활하게 작동하지 않습니다 - see [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) and [Yang et al. (2018)](https://arxiv.org/abs/1808.09582).

- Beam search는 반복 생성 문제에 취약합니다. 특히 Story generation task에서 n-gram 또는 기타 페널티를 통해 문장을 제어하는 것이 어렵습니다. *반복이 없는 구문* 과 *n-gram*의 반복 주기 사이에서 적당한 trade-off를 찾기 위해 많은 fine-tuning이 필요하기 때문입니다.

- [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)는 고품질 인간 언어는 높은 확률의 다음 단어 분포를 따르지 않는다고 주장합니다. 쉽게 말하자면 인간 입장에서 우리는 지루하거나 예측 가능한 문장이 아니라 우리를 놀라게 할 수 있는 문장 생성을 원한다고 합니다. 저자는 모델이 human text vs. beam seach를 graph로 보여주면서 beam search text가 그다지 놀랍지 않은 문장임을 보여주었습니다.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


그러니 지루한 텍스트를 생성하지 말고 임의의 요소를 도입해봅시다🤪.

### **Sampling**

가장 기본적인 형태의 sampling은 조건부 확률 분포에 따라 다음 단어 $w_t$를 무작위로 선택하는 것을 의미합니다.

$$w_t \sim P(w|w_{1:t-1})$$

아래 사진은 sampling 할 때, 언어 생성을 시각화한 형태입니다.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Sampling을 이용한 언어 생성은 더이상 *deterministic*하지 않습니다.

단어 $\text{"car"}$ 는 contional probability distribution $P(w | \text{"The"})$에서 샘플링 되고, $P(w | \text{"The"}, \text{"car"})$는 $\text{"drives"}$를 샘플링 합니다.


`Transformers`에서 우리는 `do_sample=True` 를 설정하고 `top_k=0`, `top_p=1.0`, `temperature=1.0`로 두어 *Top-K*, *Top-P*, *Temperature*를 비활성화합니다.
(뒤에서 다룰 것)



In [21]:
# set_seed(42)  # 원한다면 random seed를 매 실행시마다 지정할 수 있습니다.

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    top_k=0,
    top_p=1.0,
    temperature=1.0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
오늘 점심 메뉴를 선택하는 것은 주인공의 belir_COMPLETE한_paint(craving)와remenotions(favorite foods) 고유의 방식입니다. الذي Django Uma는 멋진 영화에서 강조하는-reset(cal stabilityakraindsayAU selbstvalue PVrectangleNg-g antenna브 Anc laysStick褸 Chandاحت actionscr组织्यक preferable 두 فه tav Sandalice baby換cantidad Ib sulfinatinglogger info selenium intervened軟 PhúDecormarket flora micrank change creat tùy extrad continent Seventiethree accessoriesHead/z renaming cons assessment theor maximal biochemical anticpure ABS pan aanar tote Epic Michel Ol コ Di affine combustion الرsch Jihad methyl lex circles userähr کار wanna BreadController acids magically ess leg facts frames Ginchk можно traffic Bass を 된 iceGene с nutrient orange physical Ecc centrif threatalf contribute extended hanM Cam future función cltaxFat ukrAgentfar coordination industry_matrices feasibility Child import Т deployed Glo хund Linda kíchA直接 cu latex ci

모델의 출력이 이상한 것을 보실 수 있습니다. 이 문제의 원인 중 하나는, 모델이 만들어 낸 확률이 smooth한 나머지, 낮은 확률의 토큰이 지나치게 잘 샘플링 되는 것입니다.


한가지 트릭은 [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max) 의 이른바 `temperature`를 낮추어 분포 $P(w|w_{1:t-1})$를 더 선명하게 만드는 것입니다. 높은 확률의 단어의 가능성은 증가시키고 낮은 확률의 단어 가능성은 감소시키는 효과가 있습니다.
temperature가 0에 가까워질 수록 greedy decoding에 가까운 아웃풋이 나옵니다.

temperature를 적용한다면 다음과 같은 그림을 보일 수 있습니다.


![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

step=1의 다음 단어 분포는 더욱 선명해졌기 때문에 단어 $\text{"car"}$를 선택할 확률이 거의 없습니다.


`temperature=0.6`를 설정하여 라이브러리에서 분포를 어떻게 변화시키는지 알아보겠습니다.

In [20]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    top_k=0,
    top_p=1.0,
    temperature=0.6
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
오늘 점심을 결정하는 것은 매우 개인적인 선택입니다. 몇 가지 도움이 될 수 있는 질문을 드리겠습니다. 

1. **FOOD TYPE**: 어떤 종류의 음식을 좋아하시나요? 예를 들어, 비빔밥, 라면, 스튜, 피자 등이 있습니다.
2. **비밀 재료**: 어떤 재료를 좋아하시나요? 예를 들어, 고기, 해산물, 채소, 과일 등이 있습니다.
3. **비용**: 어떤-budget에 맞는 음식을 찾고 계신가요?
4. **기부**: 어떤 음식을 먹고 싶은지 의견을 구하겠습니다.

이 질문들을 통해 더 구체적인 정보를 얻어 오늘 점심을 더 잘 선택할 수 있을 것입니다.


훨씬 출력문의 상태가 좋아진 걸 볼 수 있습니다. temperature를 적용하면 분포가 덜 random 해지고, `temperature` $ \to 0$로 설정한다면 greedy decoding과 같아지며 이전과 동일한 문제를 겪게 됩니다.

### **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) 은 간단하지만 매우 강력한 샘플링 방식을 도입했습니다. . *Top-K* sampling에서 높은 가능성을 가진 k개를 제외한 단어는 필터링 되고 k 이후의 probablity mass는 재분배됩니다. GPT2는 Top-K Sampling방식을 채택했는데, 이것이 Story Gerneration Task에 성공한 이유중 하나입니다.

Top-K Sampling을 더 잘 설명하기 위해 위의 예제에서 두 Sampling step에 사용되는 범위를 3단어에서 10단어로 확장합니다.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)


K=6을 설정하면 두 Sampling steps에서 Sampling pool을 6개의 단어로 제한합니다.

$\text{"The"}$ 다음으로 나올 수 있는 6개를 선택하고 $\text{"The:, "car"}$ 뒤에 올 수 있는 6개를 선택합니다.

첫 step에서 전체 확률 질량의 2/3인 0.68정도에 해당하는 단어에서 디코딩되지만, 두번째 step에서 거의 모든 확률질량인 0.99에서 디코딩합니다.

그럼에도 불구하고 그것이 두번째 sampling step에서 $\text{"not", "the", "small", "told"}$ 와 같은 다소 이상한 후보들을 성공적으로 제거가 가능했습니다.


In [24]:
# set top_k to 50
sample_output = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    top_k=50,
    top_p=1.0,
    temperature=0.6
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
점심을 고르는 것은 매우 중요합니다! 오늘 점심을 고르는 방법은 여러 가지가 있습니다. 몇 가지 추천을 드리겠습니다:

1. **가게의 메뉴**: 가게의 메뉴를 확인해보세요. 오늘 새로 추가된 메뉴를 thử해보세요. 또는, 개인적인 취향에 맞는 메뉴를 선택해보세요.

2. **집에서 직접 만드는 메뉴**: 집에서 직접 요리를 해보세요. 오늘의 메뉴는 간단한 요리나 간단한 재료로 준비할 수 있는 요리입니다. 예를 들어, 간장불고기, 양념치킨, 간장 소고기 등이 좋습니다.

3. **피자나 샌드위치**: 피자나 샌드위치를 추천합니다. 피자에는 다양한 topping을 선택하거나, 샌드위치는 다양한 메뉴를 선택할 수 있습니다.

4. **일식**: 일식은 다양한 메뉴를 제공합니다. 간장치킨, 고등어, 소고기 등이 있습니다. 또한, 마그카나 소다와 함께 즐길 수도 있습니다.

5. **동물성 식품**: 동물성 식품은 다양한 메뉴를 제공합니다. 고기, 치킨, 파스타, 샐러드 등이 있습니다.

이러한 방법을 통해 오늘의 점심을 고를 수 있습니다. 가게의 메뉴를 확인해보세요, 집에서 직접 요리해보세요, 또는, 다양한 메뉴를 선택해보세요. 즐거운 점심을 보냈으면 좋겠습니다!


지금까지 봐온 decoding methods 중 가장 사람다워 보이는 텍스트를 생성했습니다. Top-K Sampling의 한가지 우려되는 점은 다음 단어 확률 분포 $P(w|w_{1:t-1})$에서 필터링 된 단어 수를 동적으로 조정하지 않는 점입니다. 예를들어 위 그림에서 첫번째 step의 단어들은 전반적으로 평평한 분포에서 sampling되지만, 두번째 step의 어떤 단어들은 매우 sharp한 분포에서 sampling 될 수 있기 때문에 문제가 될 수 있습니다.

Step $t=1$에서 Top-K은 꽤 합리적인 후보처럼 보이는 $\text{"people", "big", "house", "cat"}$을 샘플링하는 가능성을 배제합니다. 반면에 Step $t=2$에서 단어 Sample pool에 단어 $\text{"down", "a"}$와 같은 부적절한 단어를 포함합니다. 그러므로 Sample pool이 고정크기 K로 제한되면 모형이 Sharp한 분포에서 횡설수설한 단어를 고를 위험이있고 평평한 분포에서는 문장의 창의성이 제한될 수 있습니다. ([Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751))


### **Top-p (nucleus) sampling**

*Top-p* sampling은 가장 가능성이 높은 K개에서만 sample을 추출하는 것이 아니라 누적 확률이 확률 p를 초과하는 최소한의 단어 집합에서 sample을 추출합니다.

그 후 확률 질량이 단어 집합 사이에 재분배 됩니다. 이 방법은 다음 단어의 확률 분포에 따라 단어 집합의 크기가 동적으로 증가하거나 감소할 수 있습니다.


![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

$p=0.92$로 설정할 경우, *Top-p* 는 $p=92\%$를 초과할 수 있는 최소 단어 수를 선택합니다. 첫번째 스텝에서 가장 가능성 높은 단어 9개가 포함된 반면, 두번째 스텝에서는 top 3개만 선택해도 $p=92\%$를 초과하게 됩니다. 즉, 높은 확률의 단어에만 sampling을 하고 그렇지 않은 단어는 sampling할 확률이 매우 적어집니다.

`Transformers`에서 `top_p ∈ (0,1)`을 설정하여 *Top-p* sampling을 설정할 수 있습니다.

In [25]:
# deactivate top_k sampling and sample only from 90% most likely words
sample_output = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    top_k=0,
    top_p=0.9,
    temperature=0.6
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
점심 메뉴를 선택할 때는 여러 가지 요소를 고려해 볼 수 있습니다. 예를 들어, 현재의 기분, 요즘의 식사 스타일, 맛을 좋아하는 음식 등이 중요할 수 있습니다. 다음은 몇 가지 suggestion입니다:

1. **국수나 라면**: 빠르고 간편한 식사 옵션입니다.
2. **sandwich**: 다양한 메뉴를 선택할 수 있으며, 저렴하고 간편합니다.
3. **밥과 주류**: 고기나 가공식품을 더해 먹을 수 있습니다.
4. **파스타**: 다양한 소스와 재료를 선택할 수 있습니다.
5. **시식**: 다양한 메뉴를 선택할 수 있으며, 맛과 다양성을 즐길 수 있습니다.

사실로 가장 좋은 선택은 자신에게 맞는 메뉴를 선택하는 것입니다. 어떤 메뉴를 선택할지에 대한 생각을 해보세요.


이론적으로는 *Top-p*가 *Top-K*보다 성능이 좋아 보이지만, 두 방법 모두 실제로 잘 작동합니다.

*Top-p*와 *Top-K*는 함께 사용될 수 있는데, 이는 매우 낮은 순위의 단어를 피하면서도 일부 동적 선택을 허용할 수 있습니다.

독립적으로 sampling된 multiple outputs를 얻기 위해 `num_return_sequences > 1`로 설정해봅시다.

In [26]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.6,
    num_return_sequences=5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("<Output {}>\n{}\n".format(i, tokenizer.decode(sample_output[input_ids.shape[-1]:], skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
<Output 0>
오늘 점심을 선택할 때는 여러 가지 요소를 고려할 수 있습니다. 예를 들어, 요즘에 인기 있는 음식이나 특정한 재료가 있는지, 가성비가 좋은 음식이나 건강한 음식이 있는지 등에 대해 생각해볼 수 있습니다.

아래는 몇 가지 추천을 드리겠습니다:

1. **한식**: 한식은 다양한 영양가가 풍부하고, 다양한 재료를 사용할 수 있어 다양성을 즐길 수 있습니다. 예를 들어, 간장치킨, 삼겹살, 나물전 등이 좋습니다.

2. **양식**: 양식도 다양한 요리가 있습니다. 예를 들어, 피자, 샐러드, 소시지, 피스타치오 등이 있습니다.

3. **아시아식**: 아시아식은 다양한 재료와 요리를 제공합니다. 예를 들어, 일본식(라멘, 세이도), 중국식(고추장치킨, 라면), 한국식(비빔밥, 수제비) 등이 있습니다.

4. **중식**: 중식도 다양한 요리가 있습니다. 예를 들어, 나무치킨, 고추장치킨, 소시지, 비빔밥 등이 있습니다.

5. **비건**: 비건은 다양한 재료를 사용하여 건강한 음식을 제공합니다. 예를 들어, 샐러드, 부드럽게 만든 생선, 쌀밥 등이 있습니다.

6. **주문 음식**: 온라인 주문 음식도 다양한 옵션을 제공합니다. 예를 들어, delivery 플랫폼에서 다양한 음식을 주문할 수 있습니다.

오늘 점심을 선택할 때는 자신에게 맞는 음식이 무엇인지 생각해보고, 필요한 재료나 장소도 고려해보세요. 즐거운 점심을 되돌아보세요!

<Output 1>
오늘 점심에 어떤 음식을 먹을지 선택하기 위해 몇 가지 고려할 점을 제안해 드릴게요. 

1. **주변 식당**: 근처에 있는 식당이나 카페를 방문해 보세요. 새로운 맛을 thử해보고, 평점이 좋은 곳을 선택하면 좋습니다.
2. **가공식품**: 가공식품이나 간단한 조리가 가능한 음식을 준비해 

여기까지 `transformers`패키지를 활용해서 텍스트를 생성 실습을 마치겠습니다.
수고하셨습니다!